
<table class="ee-notebook-buttons" align="left">
    <td><a target="_parent"  href="https://github.com/MVOSlab-sdstate/AST426_Lab/blob/main/Lab%2005/AST426%20Lab%205.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_parent"  href="https://nbviewer.org/github/MVOSlab-sdstate/AST426_Lab/blob/main/Lab%2005/AST426%20Lab%205.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_parent"  href="https://colab.research.google.com/github/MVOSlab-sdstate/AST426_Lab/blob/main/Lab%2005/AST426%20Lab%205.ipynb"><img width=26px src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
    </table>

In [ ]:
!pip install earthengine-api geemap folium matplotlib

In [ ]:
# Precision Agriculture Lab: Google Earth Engine with geemap

import ee
import geemap
import folium
import matplotlib.pyplot as plt

# Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-mvoslabsdstate')

# Task 1: Load and Display NDVI for a Region
def task1_ndvi_visualization():
    # Define a region of interest (ROI)
    roi = ee.Geometry.Rectangle([-100.0, 40.0, -99.0, 41.0])

    # Load Landsat 8 collection and filter by date and ROI
    l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterDate('2020-06-01', '2020-09-01') \
        .filterBounds(roi)

    # Function to calculate NDVI
    def addNDVI(image):
        ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
        return image.addBands(ndvi)

    # Map the function over the image collection
    l8_with_ndvi = l8.map(addNDVI)

    # Create a map
    Map = folium.Map(location=[40.5, -99.5], zoom_start=8)

    # Add NDVI layer to the map
    ndvi_vis = {'min': -1, 'max': 1, 'palette': ['red', 'yellow', 'green']}
    map_id_dict = ee.Image(l8_with_ndvi.select('NDVI').mean()).getMapId(ndvi_vis)
    folium.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        overlay=True,
        name='NDVI'
    ).add_to(Map)

    # Add layer control
    folium.LayerControl().add_to(Map)

    return Map

# Task 2: Time Series Analysis of NDVI
def task2_ndvi_time_series():
    # Define a point of interest
    point = ee.Geometry.Point([-99.5, 40.5])

    # Load Landsat 8 collection and filter by date
    l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterDate('2020-01-01', '2020-12-31') \
        .filter(ee.Filter.calendarRange(1, 12, 'month')) \
        .filter(ee.Filter.lt('CLOUD_COVER', 20))  # Filter for less cloudy images

    # Function to calculate NDVI
    def addNDVI(image):
        ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
        return image.addBands(ndvi)

    # Map the function over the image collection
    l8_with_ndvi = l8.map(addNDVI)

    # Get the NDVI time series
    def getndvi(image):
        date = image.date().format('YYYY-MM-dd')
        ndvi = image.select('NDVI').reduceRegion(ee.Reducer.first(), point, 30).get('NDVI')
        return ee.Feature(None, {'ndvi': ndvi, 'date': date})

    ndvi_series = l8_with_ndvi.map(getndvi).filter(ee.Filter.notNull(['ndvi'])).getInfo()

    # Extract dates and NDVI values
    dates = [feature['properties']['date'] for feature in ndvi_series['features']]
    ndvi_values = [feature['properties']['ndvi'] for feature in ndvi_series['features']]

    # Create a time series plot
    plt.figure(figsize=(12, 6))
    plt.plot(dates, ndvi_values, 'bo-')
    plt.xlabel('Date')
    plt.ylabel('NDVI')
    plt.title('NDVI Time Series')
    plt.xticks(rotation=45)
    plt.tight_layout()

    return plt

# Task 3: Crop Classification
def task3_crop_classification():
    # Define a region of interest (ROI)
    roi = ee.Geometry.Rectangle([-100.0, 40.0, -99.0, 41.0])

    # Load Sentinel-2 collection and filter by date and ROI
    s2 = ee.ImageCollection('COPERNICUS/S2_SR') \
        .filterDate('2020-06-01', '2020-09-01') \
        .filterBounds(roi)

    # Function to calculate indices
    def addIndices(image):
        ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
        ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
        return image.addBands(ndvi).addBands(ndwi)

    # Map the function over the image collection
    s2_with_indices = s2.map(addIndices)

    # Select bands for classification
    bands = ['B2', 'B3', 'B4', 'B8', 'NDVI', 'NDWI']
    image_to_classify = s2_with_indices.select(bands).median()

    # Create sample points
    points = ee.FeatureCollection([
        ee.Feature(ee.Geometry.Point([-99.8, 40.2]), {'crop_type': 0}),
        ee.Feature(ee.Geometry.Point([-99.6, 40.4]), {'crop_type': 1}),
        ee.Feature(ee.Geometry.Point([-99.4, 40.6]), {'crop_type': 2}),
        ee.Feature(ee.Geometry.Point([-99.2, 40.8]), {'crop_type': 3})
    ])

    # Sample the input imagery to get training data
    training = image_to_classify.sampleRegions(**{
        'collection': points,
        'properties': ['crop_type'],
        'scale': 10
    })

    # Train a classifier using smileRandomForest
    classifier = ee.Classifier.smileRandomForest(numberOfTrees=10).train(training, 'crop_type', bands)

    # Classify the image
    classified = image_to_classify.classify(classifier)

    # Create a map
    Map = folium.Map(location=[40.5, -99.5], zoom_start=8)

    # Add the classified image to the map
    class_vis = {'min': 0, 'max': 3, 'palette': ['yellow', 'green', 'blue', 'red']}
    map_id_dict = classified.getMapId(class_vis)
    folium.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        overlay=True,
        name='Crop Types'
    ).add_to(Map)

    # Add layer control
    folium.LayerControl().add_to(Map)

    return Map

# Task 4: Irrigation Detection
def task4_irrigation_detection():
    # Define a region of interest (ROI)
    roi = ee.Geometry.Rectangle([-100.0, 40.0, -99.0, 41.0])

    # Load Landsat 8 collection and filter by date and ROI
    l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterDate('2020-06-01', '2020-09-01') \
        .filterBounds(roi)

    # Calculate NDWI
    def addNDWI(image):
        ndwi = image.normalizedDifference(['SR_B3', 'SR_B5']).rename('NDWI')
        return image.addBands(ndwi)

    l8_with_ndwi = l8.map(addNDWI)

    # Calculate the mean NDWI for the period
    mean_ndwi = l8_with_ndwi.select('NDWI').mean()

    # Threshold for irrigation detection
    irrigated = mean_ndwi.gt(0.1)

    # Create a map
    Map = folium.Map(location=[40.5, -99.5], zoom_start=8)

    # Add the irrigation layer to the map
    irr_vis = {'min': 0, 'max': 1, 'palette': ['yellow', 'green']}
    map_id_dict = irrigated.updateMask(irrigated).getMapId(irr_vis)
    folium.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        overlay=True,
        name='Irrigated Areas'
    ).add_to(Map)

    # Add layer control
    folium.LayerControl().add_to(Map)

    return Map

# You can call these functions to display the results
# For example:
# task1_map = task1_ndvi_visualization()
# display(task1_map)

# task2_plot = task2_ndvi_time_series()
# task2_plot.show()

# task3_map = task3_crop_classification()
# display(task3_map)

# task4_map = task4_irrigation_detection()
# display(task4_map)

In [ ]:
task1_map = task1_ndvi_visualization()
display(task1_map)

In [ ]:
task2_plot = task2_ndvi_time_series()
task2_plot.show()

In [ ]:
task3_map = task3_crop_classification()
display(task3_map)

In [ ]:
task4_map = task4_irrigation_detection()
display(task4_map)